# Training Pipeline
[run_training_pipeline.ipynb](https://github.com/shibing624/MedicalGPT/blob/main/run_training_pipeline.ipynb)    | [Open In Colab](https://colab.research.google.com/github/shibing624/MedicalGPT/blob/main/run_training_pipeline.ipynb)

# Stage 1: Continue Pretraining

第一阶段：PT(Continue PreTraining)增量预训练，在海量领域文本数据上二次预训练GPT模型，以适配领域数据分布

注意：
1. 此阶段是可选的，如果你没有海量领域文本，可以跳过此阶段，直接进行SFT阶段的有监督微调
2. 我实验发现：做领域知识注入，SFT比PT更高效，也可以跳过PT阶段

| Stage 1: Continue Pretraining   |  [pretraining.py](https://github.com/shibing624/MedicalGPT/blob/main/pretraining.py) | [run_pt.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_pt.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Qwen/Qwen2.5-0.5B
2. 数据集：PT阶段使用的是中文天龙八部小说部分文本和英文书籍部分文本，位于`data/pretrain`文件夹

## 配置运行环境

本地执行可注释以下配置环境的命令，colab执行要打开注释，用于配置环境

colab建议使用T4 GPU训练，设置方式：`代码执行程序 -> 更改运行时类型 -> 运行时类型：Python3，硬件加速器：GPU，GPU类型：T4 -> 保存`

步骤：
1. 下载最新代码到本地
2. 安装依赖包

依赖包如下，保证最新版本：

```
loguru
transformers
sentencepiece
datasets
tensorboard
tqdm
peft
trl
```

In [1]:
!git clone --depth 1 https://github.com/shibing624/MedicalGPT.git
%cd MedicalGPT
%ls
!pip install -r requirements.txt

Cloning into 'MedicalGPT'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 96 (delta 19), reused 54 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (96/96), 8.56 MiB | 22.41 MiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/MedicalGPT
build_domain_tokenizer.py   requirements.txt
chatpdf.py                  reward_modeling.py
CITATION.cff                role_play_data/
_config.yml                 run_dpo.sh
CONTRIBUTING.md             run_eval_quantize.sh
convert_dataset.py          run_full_sft.sh
data/                       run_grpo.sh
DISCLAIMER                  run_orpo.sh
docs/                       run_ppo.sh
dpo_training.py             run_pt.sh
eval_quantize.py            run_quant.sh
fastapi_server_demo.py      run_rm.sh
gradio_demo.py              run_sft_accelerate.sh
grpo_training.py            run_sft.sh
inference_multigpu_demo.py  run_training_dpo_pipel

In [2]:
!pip install fsspec==2025.3.2
!pip install antlr4-python3-runtime==4.9.*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=d97de38fd66b3fb504bd7bb61942184e0ee601267cfaee3bf3fe0ff0b9e6c383
  Stored in directory: /root/.cache/pip/wheels/1a/97/32/461f837398029ad76911109f07047fde1d7b661a147c7c56d1
Successfully built antlr4-python3-runtime
  Attempting unins

In [3]:
!pip install -r requirements.txt --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.9 MB/s eta 0:00:00
  Using cached latex2sympy2_extended-1.10.2-py3-none-any.whl.metadata (5.3 kB)
  Using cached antlr4_python3_runtime-4.13.2-py3-none-any.whl.metadata (304 bytes)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached antlr4_python3_runtime-4.13.2-py3-none-any.whl (144 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 42.8 MB/s eta 0:00:00
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.9.3
    Uninstalling antlr4-python3-runtime-4.9.3:
      Successfully uninstalled antlr4-python3-runtime-4.9.3
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
    

In [4]:
!pip show transformers

Name: transformers
Version: 4.53.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers, trl


In [5]:
!pip install transformers==4.49.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 49.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.2
    Uninstalling transformers-4.53.2:
      Successfully uninstalled transformers-4.53.2


In [16]:
!pip show tqdm
!pip show trl
!pip show datasets
!pip show peft

Name: tqdm
Version: 4.47.0
Summary: Fast, Extensible Progress Meter
Home-page: https://github.com/tqdm/tqdm
Author: 
Author-email: 
License: MPLv2.0, MIT Licences
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: bigquery-magics, cmdstanpy, dataproc-spark-connect, datasets, dopamine_rl, gdown, google-generativeai, huggingface-hub, hyperopt, ipyparallel, kaggle, kagglehub, moviepy, nltk, openai, panel, peft, proglog, prophet, sentence-transformers, shap, spacy, tensorflow-datasets, torchtune, transformers, tsfresh, umap-learn
Name: trl
Version: 0.15.2
Summary: Train transformer language models with reinforcement learning.
Home-page: https://github.com/huggingface/trl
Author: Leandro von Werra
Author-email: leandro.vonwerra@gmail.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: accelerate, datasets, rich, transformers
Required-by: 
Name: datasets
Version: 2.16.0
Summary: HuggingFace community-driven open-source library of dataset

In [18]:
!pip install datasets==2.21.0
!pip install tqdm==4.67


  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1


## Stage1 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

**以下参数可以根据你的GPU实际情况修改，当前参数是根据Colab的T4单卡GPU（16GB显存）配置的**

In [19]:
%ls ./data/pretrain/

en_article_tail500.txt  fever.txt  tianlongbabu.txt


In [20]:
!python pretraining.py \
    --model_name_or_path Qwen/Qwen2.5-0.5B \
    --train_file_dir ./data/pretrain \
    --validation_file_dir ./data/pretrain \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 3 \
    --do_train \
    --do_eval \
    --use_peft True \
    --seed 42 \
    --bf16 \
    --max_train_samples 20000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --eval_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --block_size 128 \
    --group_by_length True \
    --output_dir outputs-pt-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2025-07-14 02:18:58.442459: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752459538.491095   13555 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752459538.501641   13555 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-14 02:18:58.538724: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-14 02:19:05.918 | INFO     | __main__:main:359 - Model args: ModelArguments(model_name_or_path='Qwen/Qwen2.5-

In [21]:
!pip show math-verify


Name: math-verify
Version: 0.5.2
Summary: A library for verifying mathematical answers
Home-page: 
Author: 
Author-email: Hynek Kydlíček <hynek.kydlicek@huggingface.co>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: latex2sympy2_extended
Required-by: 


In [31]:
!pip uninstall datasets
!pip install tqdm==2.14.6

Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Would remove:
    /usr/local/bin/datasets-cli
    /usr/local/lib/python3.11/dist-packages/datasets-4.0.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/datasets/*
Proceed (Y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/uninstall.py", line 106, in run
    uninstall_pathset = req.uninstall(
                        ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/req/req_install.py", line 722, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/req/req_uninstall.py", line 364, in remove
    if auto_confirm or self._allowed_to_proceed

In [22]:
%ls -lh outputs-pt-v1

total 22M
-rw-r--r-- 1 root root  901 Jul 14 02:30 adapter_config.json
-rw-r--r-- 1 root root  17M Jul 14 02:30 adapter_model.safetensors
-rw-r--r-- 1 root root  605 Jul 14 02:30 added_tokens.json
-rw-r--r-- 1 root root  470 Jul 14 02:30 all_results.json
drwxr-xr-x 2 root root 4.0K Jul 14 02:26 checkpoint-500/
drwxr-xr-x 2 root root 4.0K Jul 14 02:30 checkpoint-835/
-rw-r--r-- 1 root root  262 Jul 14 02:30 eval_results.json
-rw-r--r-- 1 root root 1.6M Jul 14 02:30 merges.txt
-rw-r--r-- 1 root root 5.1K Jul 14 02:30 README.md
drwxr-xr-x 3 root root 4.0K Jul 14 02:20 runs/
-rw-r--r-- 1 root root  616 Jul 14 02:30 special_tokens_map.json
-rw-r--r-- 1 root root 7.2K Jul 14 02:30 tokenizer_config.json
-rw-r--r-- 1 root root  20K Jul 14 02:30 trainer_state.json
-rw-r--r-- 1 root root  228 Jul 14 02:30 train_results.json
-rw-r--r-- 1 root root 3.3M Jul 14 02:30 vocab.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [23]:
!python merge_peft_adapter.py \
    --base_model Qwen/Qwen2.5-0.5B --lora_model outputs-pt-v1 --output_dir merged-pt/

2025-07-14 02:30:51.388109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752460251.507720   16690 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752460251.537335   16690 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-14 02:30:51.582351: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Namespace(base_model='Qwen/Qwen2.5-0.5B', tokenizer_path=None, lora_model='outputs-pt-v1', resize_emb=False, output_d

In [24]:
%ls -lh merged-pt/

total 958M
-rw-r--r-- 1 root root  605 Jul 14 02:31 added_tokens.json
-rw-r--r-- 1 root root  745 Jul 14 02:31 config.json
-rw-r--r-- 1 root root  117 Jul 14 02:31 generation_config.json
-rw-r--r-- 1 root root 1.6M Jul 14 02:31 merges.txt
-rw-r--r-- 1 root root 943M Jul 14 02:31 model.safetensors
-rw-r--r-- 1 root root  616 Jul 14 02:31 special_tokens_map.json
-rw-r--r-- 1 root root 7.1K Jul 14 02:31 tokenizer_config.json
-rw-r--r-- 1 root root  11M Jul 14 02:31 tokenizer.json
-rw-r--r-- 1 root root 2.7M Jul 14 02:31 vocab.json


In [25]:
%cat merged-pt/config.json

{
  "_name_or_path": "Qwen/Qwen2.5-0.5B",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.49.0",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 151936
}


Stage1 增量预训练完成。

# Stage 2: Supervised FineTuning

第二阶段：SFT(Supervised Fine-tuning)有监督微调，构造指令微调数据集，在预训练模型基础上做指令精调，以对齐指令意图，并注入领域知识

| Stage 2: Supervised Fine-tuning | [supervised_finetuning.py](https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py) | [run_sft.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_sft.sh)  |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Qwen/Qwen2.5-0.5B 或者 Stage1得到的预训练模型
2. 数据集：SFT阶段使用的是使用的是Belle的1千条抽样数据，位于`data/finetune`文件夹

## Stage2 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [26]:
%ls ./data/finetune

medical_sft_1K_format.jsonl  sharegpt_zh_1K_format.jsonl


In [28]:
!python supervised_finetuning.py \
    --model_name_or_path merged-pt \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --bf16 \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.05 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --eval_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --output_dir outputs-sft-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2025-07-14 02:43:59.927824: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752461039.963873   20011 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752461039.975240   20011 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-14 02:44:00.008303: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-14 02:44:03.642 | WARNING  | __main__:__post_init__:192 - You may set max_train_samples = -1 to run all sampl

In [29]:
%ls -lh outputs-sft-v1

total 22M
-rw-r--r-- 1 root root  893 Jul 14 02:55 adapter_config.json
-rw-r--r-- 1 root root  17M Jul 14 02:55 adapter_model.safetensors
-rw-r--r-- 1 root root  605 Jul 14 02:55 added_tokens.json
-rw-r--r-- 1 root root  429 Jul 14 02:56 all_results.json
drwxr-xr-x 2 root root 4.0K Jul 14 02:55 checkpoint-250/
-rw-r--r-- 1 root root  220 Jul 14 02:56 eval_results.json
-rw-r--r-- 1 root root 1.6M Jul 14 02:55 merges.txt
-rw-r--r-- 1 root root 5.1K Jul 14 02:55 README.md
drwxr-xr-x 3 root root 4.0K Jul 14 02:44 runs/
-rw-r--r-- 1 root root  648 Jul 14 02:55 special_tokens_map.json
-rw-r--r-- 1 root root 7.2K Jul 14 02:55 tokenizer_config.json
-rw-r--r-- 1 root root 6.0K Jul 14 02:55 trainer_state.json
-rw-r--r-- 1 root root  229 Jul 14 02:55 train_results.json
-rw-r--r-- 1 root root 3.3M Jul 14 02:55 vocab.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [30]:
!python merge_peft_adapter.py \
    --base_model merged-pt --lora_model outputs-sft-v1 --output_dir merged-sft/

2025-07-14 02:56:20.776385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752461780.798485   23151 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752461780.804959   23151 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-14 02:56:20.826119: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Namespace(base_model='merged-pt', tokenizer_path=None, lora_model='outputs-sft-v1', resize_emb=False, output_dir='mer

In [31]:
%ls -lh merged-sft/

total 958M
-rw-r--r-- 1 root root  605 Jul 14 02:56 added_tokens.json
-rw-r--r-- 1 root root  737 Jul 14 02:56 config.json
-rw-r--r-- 1 root root  117 Jul 14 02:56 generation_config.json
-rw-r--r-- 1 root root 1.6M Jul 14 02:56 merges.txt
-rw-r--r-- 1 root root 943M Jul 14 02:56 model.safetensors
-rw-r--r-- 1 root root  616 Jul 14 02:56 special_tokens_map.json
-rw-r--r-- 1 root root 7.1K Jul 14 02:56 tokenizer_config.json
-rw-r--r-- 1 root root  11M Jul 14 02:56 tokenizer.json
-rw-r--r-- 1 root root 2.7M Jul 14 02:56 vocab.json


In [32]:
%cat merged-sft/config.json

{
  "_name_or_path": "merged-pt",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.49.0",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 151936
}


Stage2 SFT训练完成。

# Stage 3: Reward Modeling

第三阶段：RM(Reward Model)奖励模型建模，构造人类偏好排序数据集，训练奖励模型，用来对齐人类偏好，主要是"HHH"原则，具体是"helpful, honest, harmless"

| Stage 3: Reward Modeling        |  [reward_modeling.py](https://github.com/shibing624/MedicalGPT/blob/main/reward_modeling.py) | [run_rm.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_rm.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Qwen/Qwen2.5-0.5B 或者 Stage2得到的SFT模型
2. 数据集：RM阶段使用的是医疗reward数据，抽样了500条，位于`data/reward`文件夹

## Stage3 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [33]:
%ls ./data/reward/

dpo_zh_500.jsonl


In [34]:
!python reward_modeling.py \
    --model_name_or_path merged-sft \
    --train_file_dir ./data/reward \
    --validation_file_dir ./data/reward \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --use_peft True \
    --seed 42 \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.001 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --eval_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --max_source_length 256 \
    --max_target_length 256 \
    --output_dir outputs-rm-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --remove_unused_columns False \
    --gradient_checkpointing True

2025-07-14 02:57:00.358108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752461820.378920   23332 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752461820.385219   23332 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-14 02:57:00.405570: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-14 02:57:03.972 | INFO     | __main__:main:332 - Model args: ModelArguments(model_name_or_path='merged-sft', 

In [35]:
%ls -lh outputs-rm-v1

total 22M
-rw-r--r-- 1 root root  923 Jul 14 03:02 adapter_config.json
-rw-r--r-- 1 root root  17M Jul 14 03:02 adapter_model.safetensors
-rw-r--r-- 1 root root  605 Jul 14 03:02 added_tokens.json
-rw-r--r-- 1 root root  485 Jul 14 03:02 all_results.json
drwxr-xr-x 2 root root 4.0K Jul 14 03:02 checkpoint-339/
-rw-r--r-- 1 root root  291 Jul 14 03:02 eval_results.json
-rw-r--r-- 1 root root 1.6M Jul 14 03:02 merges.txt
-rw-r--r-- 1 root root 5.1K Jul 14 03:02 README.md
drwxr-xr-x 3 root root 4.0K Jul 14 02:57 runs/
-rw-r--r-- 1 root root  648 Jul 14 03:02 special_tokens_map.json
-rw-r--r-- 1 root root 7.1K Jul 14 03:02 tokenizer_config.json
-rw-r--r-- 1 root root 8.4K Jul 14 03:02 trainer_state.json
-rw-r--r-- 1 root root  214 Jul 14 03:02 train_results.json
-rw-r--r-- 1 root root 3.3M Jul 14 03:02 vocab.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [36]:
!python merge_peft_adapter.py \
    --base_model merged-sft --lora_model outputs-rm-v1 --output_dir merged-rm/

2025-07-14 03:02:29.270446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752462149.291973   24804 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752462149.299903   24804 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-14 03:02:29.321157: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Namespace(base_model='merged-sft', tokenizer_path=None, lora_model='outputs-rm-v1', resize_emb=False, output_dir='mer

In [37]:
%ls -lh merged-rm/

total 1.9G
-rw-r--r-- 1 root root  605 Jul 14 03:02 added_tokens.json
-rw-r--r-- 1 root root  829 Jul 14 03:02 config.json
-rw-r--r-- 1 root root 1.6M Jul 14 03:02 merges.txt
-rw-r--r-- 1 root root 1.9G Jul 14 03:02 model.safetensors
-rw-r--r-- 1 root root  616 Jul 14 03:02 special_tokens_map.json
-rw-r--r-- 1 root root 7.1K Jul 14 03:02 tokenizer_config.json
-rw-r--r-- 1 root root  11M Jul 14 03:02 tokenizer.json
-rw-r--r-- 1 root root 2.7M Jul 14 03:02 vocab.json


In [38]:
%cat merged-rm/config.json

{
  "_name_or_path": "merged-sft",
  "architectures": [
    "Qwen2ForSequenceClassification"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "tie_word_embeddings": true,
  "torch_dtype": "float32",
  "transformers_version": "4.49.0",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 151936
}


Stage3 奖励建模第一次训练完成。

# Stage 4: Reinforcement Learning Training

第四阶段：RL(Reinforcement Learning)基于人类反馈的强化学习(RLHF)，用奖励模型来训练SFT模型，生成模型使用奖励或惩罚来更新其策略，以便生成更高质量、更符合人类偏好的文本

| Stage 4: Reinforcement Learning |  [rl_training.py](https://github.com/shibing624/MedicalGPT/blob/main/rl_training.py) | [run_rl.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_rl.sh)    |


#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型、奖励模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Qwen/Qwen2.5-0.5B 或者 Stage2得到的SFT模型
2. 奖励模型：使用的是`OpenAssistant/reward-model-deberta-v3-large-v2` 或者 Stage3得到的BERT类或者GPT类奖励模型
3. 数据集：RL阶段的数据可以复用SFT的数据集，使用的是Belle的1千条抽样数据，位于`data/finetune`文件夹

## Stage4 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载生成模型和tokenizer，加载奖励模型和其tokenizer
5. 开始训练并评估
6. 查看训练结果

以下参数可以根据你的GPU实际情况修改，当前参数是根据Colab的T4单卡GPU（16GB显存）配置的。

In [39]:
%ls ./data/finetune/

medical_sft_1K_format.jsonl  sharegpt_zh_1K_format.jsonl


In [42]:
! CUDA_VISIBLE_DEVICES=0 python ppo_training.py \
    --sft_model_path ./merged-sft \
    --reward_model_path ./merged-rm \
    --template_name qwen \
    --torch_dtype bfloat16 \
    --bf16 \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --batch_size 1 \
    --mini_batch_size 1 \
    --max_source_length 256 \
    --response_length 1000 \
    --do_train \
    --save_steps 50 \
    --output_dir outputs-ppo-v1 \
    --num_train_epochs 3 \
    --report_to tensorboard

2025-07-14 03:09:27.720108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752462567.744178   26603 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752462567.750959   26603 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-14 03:09:27.773550: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-14 03:09:33.601 | INFO     | __main__:main:56 - Parse args: PPOArguments(dataset_name=None, dataset_config=No

In [ ]:
%ls -lh outputs-ppo-v1

模型训练结果：
- use_peft=False,默认是使用全参训练，模型保存的就是`model-00001-of-00002.safetensors`等文件，配置文件是`config.json`
- use_peft=True, 则使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/trl`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/trl --host 0.0.0.0 --port 8009`

In [ ]:
%ls -lh outputs-ppo-v1/

In [ ]:
%cat outputs-ppo-v1/config.json

Stage4 RL第一次训练完成。

**至此一个完整的4阶段训练流程演示完成。**

实际操作中Stage3和Stage4可以反复多次，直到RL得到的最后模型满足评估要求。

RLHF过程可以把SFT模型当成一个初始化模型，RM模型当做指导老师，使用RL(PPO)调教SFT模型生成指导老师最满意的结果，如果小学老师满意了，我们就再训练一个中学老师，继续指导，中学老师满意了，就训练一个大学老师，这样不断迭代，使得生成模型的质量达到甚至超过人工撰写的天花板。

RLHF训练不易，此项目提供给大家一种实现的方法和参考，希望抛砖引玉，共同促进中文开源LLM发展。

# Test

In [ ]:
!python inference.py --base_model merged-ppo-v1
# 或在shell中运行
# !python inference.py --base_model merged-ppo-v1 --interactive

Input:介绍下南京
Response:  南京市位于江苏省西南部，是全国首批历史文化名城、国家中心城市和自由贸易试验区。

完。
